Last Access to dataset: 11/19/2020 at 7:30 PM

# getSpotifyData

### Packages, API, and References 

[Spotipy](https://github.com/plamere/spotipy) is a python wrapper for the [Spotify Web API](https://developer.spotify.com/documentation/web-api/). 

Referenced: https://github.com/MaxHilsdorf/introduction_to_spotipy

First, run all chunks in *Importing Packages* and *Seting up Credentials* to set up spotify web API credentials 
To find your client id and client secret check the spotify developer dashboard 
To pull data, run get_spotify_playlist_data() 

### Table of Contents 
 - Importing Packages 
 - Setting up Credentials 
 - query_spotify_playlist() function 
 - query_multiple_spotify_playlists() function 
 - query_playlist_from_categories() function
 - Pull all data and save data frames to pickle for analysis 
 - get_spotify_playlist_data() function: Run This to Pull Data 
 

#### Importing Packages 

In [1]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials


import getpass
import pickle

#### Seting up Credentials 

In [2]:
CLIENT_ID = getpass.getpass(prompt = "your-spotify-client-id")

your-spotify-client-id········


In [3]:
CLIENT_SECRET = getpass.getpass(prompt = "your-spotify-client-secret")

your-spotify-client-secret········


In [4]:
import warnings;
warnings.filterwarnings('ignore');

token = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

#### Pulling Track in Playlist Data 

In [11]:

#query_spotify_playlist: Creates a dataframe of songs and assorted metadata from a playlist.

# Parameters 
# :creator: (string) username who authored playlist 
# :playlist_id: (string) a unique primary key for the playlist (found from playlist URL extention)
# :track_feature_list: (array) a list of strings of different features to be queried in dataframe 
#                      learn more about track feautres here
#                     https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/
# :metadata_list: (array) [Default] a list of mettadata to be queried, same for all tracks

# Returns 
# :playlist_df: a pandas dataframe of the playlist 
 
def query_spotify_playlist(creator, playlist_id,track_feature_list,
                           metadata_list = ["track_id","artist","album","trackName"]):
    
    colNames = metadata_list + track_feature_list
    
    #set DataFrame for the playlist 
    playlist_df = pd.DataFrame(columns = colNames)
    
    #pull playlist data from Spotify Web API
    playlist = sp.user_playlist_tracks(creator,playlist_id)["items"]
    
    #loop over all items in a playlist 
    for i in playlist:
        playlist_elem = {}
        
        # Metadata (hardcoded locations of mettadata)
        playlist_elem["track_id"] = i["track"]["id"]
        playlist_elem["artist"] = i["track"]["album"]["artists"][0]["name"]
        playlist_elem["album"] = i["track"]["album"]["name"]
        playlist_elem["trackName"] = i["track"]["name"]
        
        # Track Audio Features
        audio_features = sp.audio_features(playlist_elem["track_id"])[0]
        for feature in track_feature_list:
            playlist_elem[feature] = audio_features[feature]
            
        # add track data to playlist dataframe     
        track_df = pd.DataFrame(playlist_elem, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df 

#### Example Playlist 
 - [Playlist Link](https://open.spotify.com/playlist/37i9dQZF1DX4jsulumEbDn)
 - Playlist Name: This is LCD Soundsystem 
 - Playlist Author: Spotify 
 - Playlist ID: 37i9dQZF1DX4jsulumEbDn
 - Playlist ID of any playlist can be found by looking at the spotify URL string 


In [8]:
playlistID_1 = "37i9dQZF1DX4jsulumEbDn"

In [18]:
#list of audo feature objects to be saved to dataframe 
audioFeatureList = ["acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","speechiness","tempo","time_signature","valence"]


playList_df_1 = query_spotify_playlist("spotify", playlistID_1,audioFeatureList)

In [19]:
playList_df_1.head()

,track_id,artist,album,trackName,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2Ud3deeqLAG988pfW0Kwcl,LCD Soundsystem,Sound of Silver,All My Friends,0.148000,0.701,462267,0.788,0.62500,2,0.0977,-5.611,1,0.0425,142.584,4,0.795
1,2cmRpmO04TLaKPzmAzySYZ,LCD Soundsystem,This Is Happening,Dance Yrself Clean,0.005570,0.739,536471,0.611,0.72500,11,0.0400,-9.829,1,0.0622,98.004,4,0.794
2,53PkA8aXiwH4ppa0V0iO7o,LCD Soundsystem,american dream,oh baby,0.019200,0.580,349693,0.622,0.65300,5,0.6480,-12.005,1,0.0352,169.442,4,0.781
3,2VGDntFPvgvqSiUf9ITEfW,LCD Soundsystem,Sound of Silver,Someone Great,0.041800,0.722,390013,0.894,0.00374,7,0.1100,-5.903,1,0.0325,113.003,4,0.396
4,1XlDNpWy8dyEljyRd0RC2J,LCD Soundsystem,LCD Soundsystem,Losing My Edge,0.000208,0.769,473067,0.959,0.63100,5,0.3310,-8.717,0,0.0634,116.168,4,0.893


In [12]:
playList_df_1.dtypes

track_id             object
artist               object
album                object
trackName            object
acousticness        float64
danceability        float64
duration_ms          object
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
dtype: object

In [13]:
audio_feature = sp.audio_features("2Ud3deeqLAG988pfW0Kwcl")

In [14]:
audio_feature

[{'danceability': 0.701,
  'energy': 0.788,
  'key': 2,
  'loudness': -5.611,
  'mode': 1,
  'speechiness': 0.0425,
  'acousticness': 0.148,
  'instrumentalness': 0.625,
  'liveness': 0.0977,
  'valence': 0.795,
  'tempo': 142.584,
  'type': 'audio_features',
  'id': '2Ud3deeqLAG988pfW0Kwcl',
  'uri': 'spotify:track:2Ud3deeqLAG988pfW0Kwcl',
  'track_href': 'https://api.spotify.com/v1/tracks/2Ud3deeqLAG988pfW0Kwcl',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2Ud3deeqLAG988pfW0Kwcl',
  'duration_ms': 462267,
  'time_signature': 4}]

In [15]:
audio_feature = sp.audio_features("76bJU0j6c1rMAS3cNeWNr5")
audio_feature

[{'danceability': 0.442,
  'energy': 0.779,
  'key': 5,
  'loudness': -5.392,
  'mode': 0,
  'speechiness': 0.0394,
  'acousticness': 0.00969,
  'instrumentalness': 0.000539,
  'liveness': 0.113,
  'valence': 0.203,
  'tempo': 100.969,
  'type': 'audio_features',
  'id': '76bJU0j6c1rMAS3cNeWNr5',
  'uri': 'spotify:track:76bJU0j6c1rMAS3cNeWNr5',
  'track_href': 'https://api.spotify.com/v1/tracks/76bJU0j6c1rMAS3cNeWNr5',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/76bJU0j6c1rMAS3cNeWNr5',
  'duration_ms': 602080,
  'time_signature': 3}]

#### Pulling Multiple Playlists 

In [22]:
#query_multiple_spotify_playlists: Takes a dictionary of playlists and returns a dictionary of pandas dataframes 

# Parameters 
# :playlists: (dict) Key -> (str) Playlist name: 
#                            [:creator: (str)username who authored playlist,
#                            playlist_id: (string) a unique primary key for the playlist (found from playlist URL extention)
#                            category_id: (string) The Spotify category ID (genera)

# :track_feature_list: (array) a list of strings of different features to be queried in dataframe 
#                      learn more about track feautres here
#                     https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/
# :metadata_list: (array) [Default] a list of mettadata to be queried, same for all tracks

# Returns 
# :playlist_dict: a dictionary of a pandas dataframe of the playlist
#                Key -> (str) Playlist name: [dataframe of tracks in playlist]


def query_multiple_spotify_playlists(playlists,track_feature_list,
                                    metadata_list = ["track_id","artist","album","trackName"], echo = False):
    
    # Create playlist_dict to hold all output playlists 
    playlist_dict = {}
    
    #Loop over every playlist in playlists  
    for playlist in playlists.items():
        
        playlist_name = playlist[0];playlist_creator = playlist[1][0];playlist_id = playlist[1][1];playlist_category_id = playlist[1][2];
        
        # Print curent playlist being pulled from API
        if echo:
            print(playlist_name + " " + playlist_id + " " + playlist_category_id)
        
        # if API unable to find playlist data, skip playlist 
        try:
            #query track data from playlist, save as dataframe 
            playlist_df = query_spotify_playlist(playlist_creator, playlist_id,track_feature_list)
        
            #lable track data with spotify category ID (genera)
            playlist_df = playlist_df.assign(category_id = playlist_category_id )
        
            #add playlsit_df to playlist_dict
            playlist_dict[playlist_name] = playlist_df
            
        except:
            print("playlist Error: Unable to query " + playlist_name + " by " + playlist_creator)
            print(playlist_id)
            continue

        
    #check if all playlists dataframes added to dictionary     
    assert len(playlists) == len(playlist_dict)
    #return dict of data_frames 
    return playlist_dict
    

In [61]:
audioFeatureList = ["acousticness","danceability","duration_ms","energy","instrumentalness",
                    "key","liveness","loudness","mode","speechiness","tempo","time_signature","valence"]

test_playlist_dict = {
    "This Is: The Mountain Goats" : ["spotify", "37i9dQZF1DWTmUQ5ZV8Wa1","category"],
    "Genre Glitch" : ["spotify", "37i9dQZF1DWUraJYejk11q","category"],
    "Deep Focus" : ["spotify", "37i9dQZF1DWZeKCadgRdKQ","category" ]
}

In [62]:
testPlaylistDict = query_multiple_spotify_playlists(test_playlist_dict,audioFeatureList)

In [63]:
testPlaylistDict["This Is: The Mountain Goats"].head()

,track_id,artist,album,trackName,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,category_id
0,3zmDm22r3qPLX9K91sagh2,The Mountain Goats,Goths,Rain in Soho,0.34300,0.633,286213,0.796,0.826,6,0.1040,-9.095,0,0.0436,127.685,4,0.4120,category
1,3tQSfujPolA1NBmv6hzcxs,The Mountain Goats,Goths,Wear Black,0.15200,0.701,250693,0.436,0.616,1,0.0965,-12.374,1,0.0338,143.022,4,0.5960,category
2,3ImifXwRr0kej9hlD67O3q,The Mountain Goats,Goths,Shelved,0.21000,0.625,241053,0.636,0.619,7,0.2610,-11.279,1,0.0291,123.054,4,0.2340,category
3,7hyfYgXMS5BumjAjY98Xoe,The Mountain Goats,Beat the Champ,The Legend of Chavo Guerrero,0.00771,0.688,180133,0.858,0.274,11,0.3680,-5.803,0,0.0449,142.117,4,0.9140,category
4,3Tar9LuKp0dx6nOefqqRMv,The Mountain Goats,Beat the Champ,Heel Turn 2,0.32300,0.494,358253,0.465,0.064,7,0.0761,-7.611,1,0.0468,154.961,4,0.0501,category


In [64]:
testPlaylistDict["Deep Focus"].head()

,track_id,artist,album,trackName,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,category_id
0,7JrSIPcfkWhDzxWII8Jz7V,Benjamin Martins,Floating,Escaping Time,0.640,0.320,200346,0.2560,0.862,10,0.112,-17.980,1,0.0445,115.011,4,0.1910,category
1,62X7ld1sa8RHl4zRtSvfHf,369,432 Water Crystals,Just Look at You,0.985,0.558,187386,0.2490,0.925,7,0.103,-14.715,1,0.0303,94.991,4,0.1110,category
2,06G5MnuMA3cLbNwjFs8zIS,Lucas Delphy,Flowing,Flowing,0.896,0.200,131116,0.1380,0.929,0,0.207,-20.417,1,0.0301,71.631,4,0.1380,category
3,51c5Wbzs0fPu6mwpDn3bsQ,Jacob Yoffee,With Resolve,With Resolve,0.982,0.456,129762,0.0195,0.924,9,0.106,-33.594,1,0.0400,126.086,4,0.2690,category
4,5AciiCKJKTLKRrAPSrKQy5,Amaranth Cove,Infinite Sustain,Infinite Sustain,0.800,0.529,149000,0.2140,0.907,6,0.109,-19.268,1,0.0315,124.069,4,0.0903,category


#### query_playlist_from_categories

In [13]:
#query_playlist_from_categories: Returns large dictionary of X number of playlists from Y number of categories 

# Parameters 
# :limit = 25: (int) number of playlist pulled for each category 
# :spotify_categories (list): list of spotify category id's 

# Returns 
# :playlist_dict: a dictionary of a pandas dataframe of the playlists 
#                Key -> (str) Playlist name: [:creator: (str)username who authored playlist,
#                            playlist_id: (string) a unique primary key for the playlist (found from playlist URL extention)
#                            category_id: (string) The Spotify category ID (genera)]

def query_playlist_from_categories(spotify_categories,playlist_limit = 25):
    
    # define playlist_dict
    playlist_dict = {}
    
    for cat_id in spotify_categories:
        #get playlists of given category(genre)
        playlists_raw_API = sp.category_playlists(cat_id, limit = playlist_limit)
        
        playlists = playlists_raw_API['playlists']['items']
        
        #loop over all playlists 
        for playlist in playlists:
            
            #add playlist to dict
            playlist_dict[playlist['name']] = [playlist['owner']['display_name'],
                                              playlist['id'],
                                              cat_id]
            
    return playlist_dict
            
        


Test Function by pulling two playlists from each spotify category

In [160]:
test_cat_list = ["hiphop","pop","country","rock","edm_dance","indie_alt","rnb",
                 "classical","jazz","punk","kpop","metal"]
test_all_cat = query_playlist_from_categories(test_cat_list, playlist_limit = 2)

In [161]:
test_all_cat

{'RapCaviar': ['Spotify', '37i9dQZF1DX0XUsuxWHRQd', 'hiphop'],
 'Most Necessary': ['Spotify', '37i9dQZF1DX2RxBh64BHjQ', 'hiphop'],
 "Today's Top Hits": ['Spotify', '37i9dQZF1DXcBWIGoYBM5M', 'pop'],
 'Pop Rising': ['Spotify', '37i9dQZF1DWUa8ZRTfalHk', 'pop'],
 'Hot Country': ['Spotify', '37i9dQZF1DX1lVhptIYRda', 'country'],
 "90's Country": ['Spotify', '37i9dQZF1DWVpjAJGB70vU', 'country'],
 'Rock Classics': ['Spotify', '37i9dQZF1DWXRqgorJj26U', 'rock'],
 'Rock & Roll Party': ['Spotify', '37i9dQZF1DWYE5MI4mMuii', 'rock'],
 'mint': ['Spotify', '37i9dQZF1DX4dyzvuaRJ0n', 'edm_dance'],
 'Chill Tracks': ['Spotify', '37i9dQZF1DX6VdMW310YC7', 'edm_dance'],
 'Alternative 10s': ['Spotify', '37i9dQZF1DX873GaRGUmPl', 'indie_alt'],
 'Alternative 90s': ['Spotify', '37i9dQZF1DXaLI8a395lse', 'indie_alt'],
 "I Love My '00s R&B": ['Spotify', '37i9dQZF1DWYmmr74INQlb', 'rnb'],
 "I Love My '90s R&B": ['Spotify', '37i9dQZF1DX6VDO8a6cQME', 'rnb'],
 'Classical Essentials': ['Spotify', '37i9dQZF1DWWEJlAGA9gs0',

#### Gather All Spotify Data 

In [163]:
spotify_catigory_list = ["hiphop","pop","country","rock","edm_dance","indie_alt","rnb",
                 "classical","jazz","punk","kpop","metal"]
audioFeatureList = ["acousticness","danceability","duration_ms","energy","instrumentalness",
                    "key","liveness","loudness","mode","speechiness","tempo","time_signature","valence"]


Gather dictonary of playlists for all catigories listed in spotify_catigory_list 

In [164]:
playlist_id_dict = query_playlist_from_categories(spotify_catigory_list)

In [165]:
playlist_id_dict

{'RapCaviar': ['Spotify', '37i9dQZF1DX0XUsuxWHRQd', 'hiphop'],
 'Most Necessary': ['Spotify', '37i9dQZF1DX2RxBh64BHjQ', 'hiphop'],
 'Signed XOXO': ['Spotify', '37i9dQZF1DX2A29LI7xHn1', 'hiphop'],
 'Gold School': ['Spotify', '37i9dQZF1DWVA1Gq4XHa6U', 'hiphop'],
 "Feelin' Myself": ['Spotify', '37i9dQZF1DX6GwdWRQMQpq', 'hiphop'],
 'I Love My Down South Classics': ['Spotify',
  '37i9dQZF1DWYok9l1JL7GM',
  'hiphop'],
 "I Love My '80s Hip-Hop": ['Spotify', '37i9dQZF1DX2XmsXL2WBQd', 'hiphop'],
 'Black Lives Matter': ['Spotify', '37i9dQZF1DWWAqc46ZJdZf', 'rnb'],
 "Today's Top Hits": ['Spotify', '37i9dQZF1DXcBWIGoYBM5M', 'pop'],
 'Pop Rising': ['Spotify', '37i9dQZF1DWUa8ZRTfalHk', 'pop'],
 'Hot Country': ['Spotify', '37i9dQZF1DX1lVhptIYRda', 'country'],
 "90's Country": ['Spotify', '37i9dQZF1DWVpjAJGB70vU', 'country'],
 'Country Kind of Love': ['Spotify', '37i9dQZF1DX8WMG8VPSOJC', 'country'],
 'Country Gold': ['Spotify', '37i9dQZF1DWYnwbYQ5HnZU', 'country'],
 "Country's Greatest Hits:  The '80s

Gather all tracks and metadata from the playlists in playlist_id_dict

In [169]:
Spotify_playlist_data_dict = query_multiple_spotify_playlists(playlist_id_dict,audioFeatureList,echo = True)

RapCaviar
Most Necessary
Signed XOXO
Gold School
Feelin' Myself
I Love My Down South Classics
I Love My '80s Hip-Hop
Black Lives Matter
Today's Top Hits
Pop Rising
Hot Country
90's Country
Country Kind of Love
Country Gold
Country's Greatest Hits:  The '80s
Country's Greatest Hits:  The '70's
Texas Music Now
Country's Greatest Hits:  The '60s
Wild Country
Country's Greatest Hits:  The '50s
Very Nearly Nashville
Country's Greatest Hits:  The '40's
New Traditions: Country Heroes
Rock Classics
Rock & Roll Party
80s Summer Rock
90s Summer Rock
00s Rock Anthems
90s Rock Anthems
Rock This
Acoustic Rock
Grunge Forever
Modern Chill Rock
Rock Solid
10s Rock Anthems
Raised on Rock
Post-Grunge
Classic Rock Now
All New Rock
Alternative Rock's Most Precious Gems
memories
Rock School
mint
Chill Tracks
Hype
Dance Classics
Power Hour
Housewerk
Night Rider
Dance Rising
Shuffle Syndrome
Friday Cratediggers
Trance Mission
metropolis
90s House Classics
mint Acoustic
Detroit Techno
Women of Electronic
Glob

In [172]:
#backup spotify_playlist_data_dict
file_to_write = open("spotify_dict.pickle", "wb")
pickle.dump(Spotify_playlist_data_dict,file_to_write)

In [173]:
# takes the dictornary of data and combines it into one dataframe with a new col of previous key

spotify_df = pd.concat(Spotify_playlist_data_dict)
    

In [176]:
spotify_df.shape

(9767, 18)

In [184]:
spotify_df.dtypes

track_id             object
artist               object
album                object
trackName            object
acousticness        float64
danceability        float64
duration_ms          object
energy              float64
instrumentalness     object
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
category_id          object
dtype: object

#### Save spotify_df to csv and pickle 

In [177]:
spotify_df.to_pickle("spotify_df.pkl")

In [178]:
spotify_df.to_csv("spotify_df.csv")

In [4]:
#test Loading Data 
test_pickled_df = pd.read_pickle("spotify_df.pkl")

In [6]:
test_pickled_df.head()

track_id       artist  \
RapCaviar 0  7o4gBbTM6UBLkOYPw9xMCz          DDG   
          1  1RSzyxqtIO4yX3EyiV4zT5           CJ   
          2  4DuUwzP4ALMqpquHU0ltAB  Jack Harlow   
          3  0PvFJmanyNQMseIFrU708S    Pop Smoke   
          4  5SWnsxjhdcEDc7LJjq9UHk    21 Savage   

                                                         album  \
RapCaviar 0  Moonwalking in Calabasas (feat. Blueface) [Remix]   
          1                                            Whoopty   
          2                                        Tyler Herro   
          3               Shoot For The Stars Aim For The Moon   
          4                                     SAVAGE MODE II   

                                                     trackName  acousticness  \
RapCaviar 0  Moonwalking in Calabasas (feat. Blueface) - Remix       0.06140   
          1                                            Whoopty       0.19400   
          2                                        Tyler Herro       0.11000   
          3            For The Night (feat. Lil Baby & DaBaby)       0.11400   
          4                                             Runnin       0.00748   

             danceability duration_ms  energy instrumentalness key  liveness  \
RapCaviar 0         0.928      170233   0.341                0  11     0.162   
          1         0.714      123480   0.609                0   3     0.126   
          2         0.794      156498   0.756                0   5     0.247   
          3         0.823      190476   0.586                0   6     0.193   
          4         0.819      195906   0.626            0.101  10     0.167   

             loudness mode  speechiness    tempo time_signature  valence  \
RapCaviar 0   -10.196    0        0.103  129.966              4    0.745   
          1    -7.683    0        0.170  140.206              4    0.778   
          2    -7.160    0        0.136  123.066              4    0.775   
          3    -6.606    0        0.200  125.971              4    0.347   
          4    -4.574    0        0.202  143.010              4    0.415   

            category_id  
RapCaviar 0      hiphop  
          1      hiphop  
          2      hiphop  
          3      hiphop  
          4      hiphop

#### get_spotify_playlist_data

In [23]:
# get_spotify_playlist_data() Function that will save get playlist data from spotify playlsit 
#                            data to a csv and pickle 

# Parameters
# :spotify_catigory_list: (array) list of spotify category id's 
# :track_feature_list: (array) a list of strings of different features to be queried in dataframe 
#                      learn more about track feautres here
#                     https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/
# :n = 25: (int) how many playlist to save for each spotify category  
# :verbose = False: (bool) print out curent playlist being queried + playlist_id_ and playlist catigory 

# Returns
# :sucessful:  (bool) True if all playlists queried 
# Saves the dataframe of spotify data as 'Data/spotify_df.pkl' and 'Data/spotify_df.csv' 


def get_spotify_playlist_data(spotify_catigory_list, track_feature_list, n = 25, verbose = False):
    
    #Gather dictonary of playlists for all catigories listed in spotify_catigory_list 
    playlist_id_dict = query_playlist_from_categories(spotify_catigory_list, playlist_limit = n)
    
    #Gather all tracks and metadata from the playlists in playlist_id_dict
    Spotify_playlist_data_dict = query_multiple_spotify_playlists(playlist_id_dict,track_feature_list,echo = verbose)
    
    # takes the dictornary of data and combines it into one dataframe with a new col of previous key
    spotify_df = pd.concat(Spotify_playlist_data_dict)
    
    spotify_df.to_pickle("spotify_df.pkl")
    spotify_df.to_csv("spotify_df.csv")
    
    return True 


In [ ]:
####### RUN THIS TO GET DATA, MUST SET SPOTIFY API CREDENTIALS AT TOP OF NOTEBOOK####

default_catigory = ["rock","pop","country","hiphop","edm_dance","indie_alt","rnb",
                 "classical","jazz","punk","kpop","metal"]
default_features = ["acousticness","danceability","duration_ms","energy","instrumentalness",
                    "key","liveness","loudness","mode","speechiness","tempo","time_signature","valence"]


get_spotify_playlist_data(default_catigory,default_features, n = 30, verbose = True)